In [1]:
from ginga.AstroImage import AstroImage
import numpy as np
import fadvise as fa
from time import sleep

In [2]:
# It might be useful to run the following code for both
# PyFits and cfitsio. There may be a drastic time difference...
from ginga.util import io_fits as io
print io.have_fitsio
print io.have_pyfits
print io.get_fitsloader()
# io.use('astropy') # 'astropy' or 'fitsio'
#print io.get_fitsloader()

False
True


# Loading Functions

In [3]:
def load_normal(path):
    '''The base test for loading images. 
    Should the one to compare against all optimizations'''
    sleep(2)
    image = AstroImage()
    image.load_file(path)
    sum = np.sum(image.get_data())
    fa.dontneed(path) # advise os to remove from cache
    return sum

In [4]:
def load_with_advise(path):
    '''Advises with no special information about the file.
    Generic advise function'''
    fa.normal(path)
    sleep(2)
    image = AstroImage()
    image.load_file(path)
    sum = np.sum(image.get_data())
    fa.dontneed(path) # advise os to remove from cache
    return sum

In [5]:
def load_with_noreuse(path):
    '''Advises OS that the file will only be accessed once.
    Shouldn't show a different in speed, but should be tested'''
    fa.noreuse(path)
    sleep(2)
    image = AstroImage()
    image.load_file(path)
    sum = np.sum(image.get_data())
    fa.dontneed(path) # advise os to remove from cache
    return sum

In [6]:
def load_with_sequential(path): 
    '''Advises the OS that the data will be accessed in a 
    sequential fashion. Not sure how valid this assumption 
    is for loading image files.''' 
    fa.sequential(path) 
    sleep(2) 
    image = AstroImage() 
    image.load_file(path) 
    sum = np.sum(image.get_data()) 
    fa.dontneed(path) # advise os to remove from cache
    return sum

# Timings

In [7]:
# get list of functions to test
funcs = [load_normal, 
         load_with_advise, 
         load_with_noreuse, 
         load_with_sequential]

In [8]:
# get list of test images
import glob
files = glob.glob('images/*.fits')
print files

['images/scaled15000x15000.fits', 'images/5000x5000.fits', 'images/1000x1000.fits', 'images/15000x15000.fits', 'images/iris.fits', 'images/10000x10000.fits']


### First test `fitsio`

In [14]:
if io.use('fitsio'): # 'astropy' or 'fitsio', 
    # also execute only if the change passes
    print io.get_fitsloader()
    # time each function, sorted first by file
    # shows timings for each image by the different functions before 
    # moving onto testing the next image
    for f in files:
        print '```' + f
        for func in funcs:
            print '-' + func.func_name
            %time func(f)
        print '\n'

***images/scaled15000x15000.fits
-load_normal
CPU times: user 1.37 s, sys: 1.12 s, total: 2.49 s
Wall time: 5.37 s
-load_with_advise
CPU times: user 1.39 s, sys: 884 ms, total: 2.28 s
Wall time: 5.37 s
-load_with_noreuse
CPU times: user 1.34 s, sys: 832 ms, total: 2.17 s
Wall time: 6.17 s
-load_with_sequential
CPU times: user 1.48 s, sys: 708 ms, total: 2.19 s
Wall time: 6.5 s


***images/5000x5000.fits
-load_normal
CPU times: user 152 ms, sys: 176 ms, total: 328 ms
Wall time: 4.15 s
-load_with_advise
CPU times: user 132 ms, sys: 232 ms, total: 364 ms
Wall time: 5.27 s
-load_with_noreuse
CPU times: user 136 ms, sys: 216 ms, total: 352 ms
Wall time: 4.35 s
-load_with_sequential
CPU times: user 144 ms, sys: 216 ms, total: 360 ms
Wall time: 4.32 s


***images/1000x1000.fits
-load_normal
CPU times: user 12 ms, sys: 4 ms, total: 16 ms
Wall time: 2.03 s
-load_with_advise
CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 2.03 s
-load_with_noreuse
CPU times: user 8 ms, sys: 8 ms, total

### Next test `astropy.pyfits`

In [10]:
if io.use('astropy'): # 'astropy' or 'fitsio', 
    # also execute only if the change passes
    print io.get_fitsloader()
    # time each function, sorted first by file
    # shows timings for each image by the different functions before 
    # moving onto testing the next image
    for f in files:
        print '```' + f
        for func in funcs:
            print '-' + func.func_name
            %time func(f)
        print '\n'

***images/scaled15000x15000.fits
-load_normal
CPU times: user 1.57 s, sys: 588 ms, total: 2.16 s
Wall time: 5.27 s
-load_with_advise
CPU times: user 1.55 s, sys: 600 ms, total: 2.15 s
Wall time: 5.14 s
-load_with_noreuse
CPU times: user 1.6 s, sys: 628 ms, total: 2.23 s
Wall time: 5.09 s
-load_with_sequential
CPU times: user 1.52 s, sys: 572 ms, total: 2.1 s
Wall time: 5.33 s


***images/5000x5000.fits
-load_normal
CPU times: user 240 ms, sys: 100 ms, total: 340 ms
Wall time: 2.69 s
-load_with_advise
CPU times: user 180 ms, sys: 16 ms, total: 196 ms
Wall time: 2.2 s
-load_with_noreuse
CPU times: user 180 ms, sys: 12 ms, total: 192 ms
Wall time: 2.19 s
-load_with_sequential
CPU times: user 192 ms, sys: 0 ns, total: 192 ms
Wall time: 2.19 s


***images/1000x1000.fits
-load_normal
CPU times: user 24 ms, sys: 8 ms, total: 32 ms
Wall time: 2.04 s
-load_with_advise
CPU times: user 28 ms, sys: 0 ns, total: 28 ms
Wall time: 2.03 s
-load_with_noreuse
CPU times: user 24 ms, sys: 0 ns, total: 24 

### Initial Observations

1. Overall, there seems to be some differences between `load_normal` and consequent function calls. Check whether this time difference is due to caching (first call is slower, but later calls are faster and vice versa) or just better `advising`.

2. `15000x15000.fits` seems to improve considerably under advising with `pyfits`; but check whether this is consequence of how we are testing it. (see #1) Important to note that this file is the largest (in terms of disk space). 

3. For small images, results are nearly the same/insignifcant for both `PyFits`/`fitsio` and advising operations. Remember that we are putting a two second delay in these loading operations, so subtracting two from the result shows a nearly instant loading time. First impression is that in the small file size regime, image loading optimizations do not matter. 

#### Now test #1; Checking whether first call caches file, making next calls faster

In [11]:
funcs2 = funcs[::-1] # reverse it
funcs2

[<function __main__.load_with_sequential>,
 <function __main__.load_with_noreuse>,
 <function __main__.load_with_advise>,
 <function __main__.load_normal>]

##### Test `fitsio` library

In [12]:
if io.use('fitsio'): # 'astropy' or 'fitsio', 
    # also execute only if the change passes
    print io.get_fitsloader()
    # time each function, sorted first by file
    # shows timings for each image by the different functions before 
    # moving onto testing the next image
    for f in files:
        print '```' + f
        for func in funcs2:
            print '-' + func.func_name
            %time func(f)
        print '\n'

***images/scaled15000x15000.fits
-load_with_sequential
CPU times: user 1.21 s, sys: 1.27 s, total: 2.48 s
Wall time: 5.52 s
-load_with_noreuse
CPU times: user 1.3 s, sys: 832 ms, total: 2.13 s
Wall time: 4.91 s
-load_with_advise
CPU times: user 1.43 s, sys: 748 ms, total: 2.18 s
Wall time: 5.28 s
-load_normal
CPU times: user 1.32 s, sys: 656 ms, total: 1.98 s
Wall time: 4.8 s


***images/5000x5000.fits
-load_with_sequential
CPU times: user 140 ms, sys: 132 ms, total: 272 ms
Wall time: 2.59 s
-load_with_noreuse
CPU times: user 128 ms, sys: 156 ms, total: 284 ms
Wall time: 2.68 s
-load_with_advise
CPU times: user 104 ms, sys: 176 ms, total: 280 ms
Wall time: 2.67 s
-load_normal
CPU times: user 124 ms, sys: 176 ms, total: 300 ms
Wall time: 2.69 s


***images/1000x1000.fits
-load_with_sequential
CPU times: user 12 ms, sys: 16 ms, total: 28 ms
Wall time: 2.04 s
-load_with_noreuse
CPU times: user 20 ms, sys: 4 ms, total: 24 ms
Wall time: 2.04 s
-load_with_advise
CPU times: user 20 ms, sys: 4

##### Test `astropy.pyfits` library

In [13]:
if io.use('astropy'): # 'astropy' or 'fitsio', 
    # also execute only if the change passes
    print io.get_fitsloader()
    # time each function, sorted first by file
    # shows timings for each image by the different functions before 
    # moving onto testing the next image
    for f in files:
        print '```' + f
        for func in funcs2:
            print '-' + func.func_name
            %time func(f)
        print '\n'

***images/scaled15000x15000.fits
-load_with_sequential
CPU times: user 1.66 s, sys: 600 ms, total: 2.26 s
Wall time: 5.78 s
-load_with_noreuse
CPU times: user 1.54 s, sys: 548 ms, total: 2.09 s
Wall time: 4.97 s
-load_with_advise
CPU times: user 1.54 s, sys: 584 ms, total: 2.12 s
Wall time: 5.33 s
-load_normal
CPU times: user 1.59 s, sys: 524 ms, total: 2.11 s
Wall time: 5.13 s


***images/5000x5000.fits
-load_with_sequential
CPU times: user 224 ms, sys: 152 ms, total: 376 ms
Wall time: 3.56 s
-load_with_noreuse
CPU times: user 184 ms, sys: 16 ms, total: 200 ms
Wall time: 2.2 s
-load_with_advise
CPU times: user 160 ms, sys: 32 ms, total: 192 ms
Wall time: 2.19 s
-load_normal
CPU times: user 184 ms, sys: 12 ms, total: 196 ms
Wall time: 2.19 s


***images/1000x1000.fits
-load_with_sequential
CPU times: user 20 ms, sys: 8 ms, total: 28 ms
Wall time: 2.04 s
-load_with_noreuse
CPU times: user 24 ms, sys: 0 ns, total: 24 ms
Wall time: 2.03 s
-load_with_advise
CPU times: user 24 ms, sys: 0 ns

#### Revisiting Observations

1. There is a pattern to the first loading function taking longer than the subsequent calls, regardless of the function used. This implies that there might be some OS optimization that is not regulated by `python-fadvise` or the way we're clearing the file out of cache is not good enough. Unfortunately, this makes testing harder to judge. 

2. On the opposite end, advising a sequential access to `fitsio` may result in faster loading times with very large files. Look at `10000x10000.fits` and `15000x15000.fits` for examples of this. This trend is missing from the first run, where sequential access was tested last, hence this should be tested more in order to say anything conclusive. My guess is that the file might start have been cached intially or something to that effect, speeding up the first load. 

3. The subsequent loading of `15000x15000.fits` is strangely cached not by the OS, but by the `pyfits` library. When loading with `pyfits`, the first call takes ~8 secs but later calls take ~3 secs. This does not happen with `cfitsio` (constant ~8 secs regardless of advising or order of loading functions). Possible topic to investigate. Same behavior occurs with `10000x10000.fits`. 

## Conclusions #1

In all, there seems to be no significant improvement with using the python library `fadvise`. There was a bias in which the first loading function was performing an OS optimization for subsequent calls, which we determined to be independent of the call used. Upon recognizing this bias, there was no discernable difference between normally loading and advising the OS to pre-cache. 

These timings were made on an eight-core Ubuntu 15.10 running a Linux 4.2.0-36-generic kernel with eight gibabytes of memory using a SSD. As such, timings on other machines may vary. 

## Further work

1. Test with longer delay, see if this helps the OS adjust a bit more to the advise. 

2. Also perhaps test with a delay after each loading function to see if the file is cleared out the cache in that time. 

3. Find out why and how `pyfits` is optimizing reusage of `15000x15000.fits`. This is unexpected behavior. 